# Hybrid Score combination: RP3beta + UserKNN

## Import

In [1]:
## Allow more than one output for a single code cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import pandas as pd
import scipy.sparse as sps
import numpy as np
import os

from skopt.space import Real, Integer, Categorical

## Set the numpy random seed
SEED = 42
np.random.seed(SEED)

In [4]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

from Evaluation.Evaluator import EvaluatorHoldout

from Recommenders.Recommender_import_list import *

Tensorflow is not available


In [4]:
## Utility Functions
from Dataset.load_data import load_data
from Dataset.write_submission import write_submission
from Dataset.load_test_user_array import load_test_user_array

## Data Loading and Split

In [5]:
URM_all, ICM_dict = load_data()

In [ ]:
#URM_aug_train, URM_aug_validation = get_splitted_data(URM_all, ICM_dict, augmented_urm = True, split_urm_only = True)
#URM_train, URM_validation = get_splitted_data(URM_all, ICM_dict, augmented_urm = False, split_urm_only = True)
    
#evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], exclude_seen = True)

In [6]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
"""
URM_aug_train = sps.vstack([URM_train.tocoo(), 
                            #ICM_dict['genre'].T.tocoo(),
                            ICM_dict['ICM_subgenre'].T.tocoo(), 
                            #ICM_dict['event'].T.tocoo(), 
                            ICM_dict['ICM_channel'].T.tocoo()], format='csr')
"""

"\nURM_aug_train = sps.vstack([URM_train.tocoo(), \n                            #ICM_dict['genre'].T.tocoo(),\n                            ICM_dict['ICM_subgenre'].T.tocoo(), \n                            #ICM_dict['event'].T.tocoo(), \n                            ICM_dict['ICM_channel'].T.tocoo()], format='csr')\n"

In [9]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], exclude_seen = True)

EvaluatorHoldout: Ignoring 13646 ( 0.0%) Users that have less than 1 test interactions


In [10]:
test_UserID_array = load_test_user_array()

## Optimize the RP3 over the augmented URM and then use it with UserKNN

## Setup optimization for hybrid

## Look at the sparsity of the ICMs

In [9]:
output_folder_path = "result_experiments/RP3beta_augmented_subgenre_channel/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

n_cases = 100  # 50 with 30% random is a good number
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [ ]:
from functools import partial
import os, multiprocessing

from HyperparameterTuning.run_hyperparameter_search import runHyperparameterSearch_Collaborative
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

runHyperparameterSearch_Collaborative(RP3betaRecommender,
                                      URM_train = URM_aug_train,
                                      URM_train_last_test = None,
                                      metric_to_optimize = metric_to_optimize,
                                      cutoff_to_optimize = cutoff_to_optimize,
                                      n_cases = n_cases,
                                      n_random_starts = n_random_starts,
                                      evaluator_validation_earlystopping = evaluator_validation,
                                      evaluator_validation = evaluator_validation,
                                      evaluator_test = None,
                                      output_folder_path = output_folder_path,
                                      resume_from_saved = True,
                                      similarity_type_list = None,
                                      parallelizeKNN = True)

## Optimize the RP3 over the augmented URM and then use it with UserKNN

## Setup optimization for hybrid

## Look at the sparsity of the ICMs

In [10]:
output_folder_path = "result_experiments/RP3beta_augmented_subgenre_channel/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

n_cases = 100  # 50 with 30% random is a good number
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [ ]:
from functools import partial
import os, multiprocessing

from HyperparameterTuning.run_hyperparameter_search import runHyperparameterSearch_Collaborative
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

runHyperparameterSearch_Collaborative(RP3betaRecommender,
                                      URM_train = URM_aug_train,
                                      URM_train_last_test = None,
                                      metric_to_optimize = metric_to_optimize,
                                      cutoff_to_optimize = cutoff_to_optimize,
                                      n_cases = n_cases,
                                      n_random_starts = n_random_starts,
                                      evaluator_validation_earlystopping = evaluator_validation,
                                      evaluator_validation = evaluator_validation,
                                      evaluator_test = None,
                                      output_folder_path = output_folder_path,
                                      resume_from_saved = True,
                                      similarity_type_list = None,
                                      parallelizeKNN = True)

In [39]:
from Recommenders.DataIO import DataIO

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(RP3betaRecommender.RECOMMENDER_NAME + "_metadata.zip")

search_metadata.keys()

dict_keys(['time_on_validation_total', 'time_on_test_avg', 'result_on_validation_df', 'time_df', 'hyperparameters_best_index', 'time_on_test_total', 'algorithm_name_recommender', 'result_on_last', 'hyperparameters_df', 'metric_to_optimize', 'time_on_validation_avg', 'cutoff_to_optimize', 'result_on_test_df', 'result_on_test_best', 'time_on_train_avg', 'time_on_train_total', 'algorithm_name_search', 'exception_list', 'time_on_last_df', 'result_on_validation_best', 'hyperparameters_best'])

In [43]:
hyperparameters_df = search_metadata["hyperparameters_df"]
hyperparameters_df.head(54)

,topK,alpha,beta,normalize_similarity
0,1009,0.599747,0.430482,False
1,1430,0.757256,0.822657,False
2,1783,0.676932,0.203678,True
3,1183,0.659265,0.441199,True
4,1391,0.963051,0.662768,False
5,1914,0.358419,0.662776,True
6,1562,0.198396,0.478845,True
7,1638,0.101867,0.532748,True
8,1227,0.443418,0.576483,True
9,1794,0.686577,0.864229,False


In [42]:
result_on_validation_df = search_metadata["result_on_validation_df"]
result_on_validation_df.head(54)

,,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,NOVELTY,AVERAGE_POPULARITY,DIVERSITY_MEAN_INTER_LIST,DIVERSITY_HERFINDAHL,COVERAGE_ITEM,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
,cutoff,,,,,,,,,,,,,,,,,,,,,,,,,
0,10,0.333492,0.335046,0.056305,0.202591,0.203379,0.617445,0.356203,0.096344,0.945609,1.099586,0.005235,0.708638,0.718887,0.971883,0.030123,0.017,0.999414,0.945055,0.0022,5.615419,0.972252,0.008627,0.451975,3.542108,0.024899
1,10,0.315767,0.31599,0.051522,0.183864,0.183941,0.525608,0.323462,0.088589,0.881982,0.967052,0.006364,0.416605,0.953068,0.9953,0.292818,0.108146,0.999414,0.881465,0.037378,9.138137,0.995678,0.146558,0.735513,2.082388,0.030269
2,10,0.3349,0.336179,0.055544,0.201246,0.201873,0.604777,0.35401,0.095285,0.942604,1.084109,0.005278,0.679769,0.774182,0.977413,0.028407,0.018052,0.999414,0.942051,0.002688,5.964339,0.977784,0.010539,0.48006,3.397808,0.025106
3,10,0.347962,0.349357,0.059156,0.210321,0.210951,0.615773,0.36667,0.10112,0.9525,1.119663,0.00537,0.646374,0.814837,0.981478,0.058752,0.031176,0.999414,0.951941,0.004206,6.471495,0.98185,0.016492,0.52088,3.23088,0.025542
4,10,0.282459,0.28257,0.043396,0.155445,0.155468,0.462322,0.284397,0.075234,0.848776,0.836769,0.006683,0.361838,0.949603,0.994953,0.323883,0.10139,0.999414,0.848278,0.046081,9.282068,0.995331,0.180682,0.747098,1.808636,0.031786
5,10,0.349076,0.350269,0.059491,0.21047,0.211004,0.604943,0.365289,0.101658,0.948395,1.109276,0.005541,0.570176,0.888164,0.98881,0.109032,0.052328,0.999414,0.947839,0.008784,7.391907,0.989185,0.03444,0.594962,2.850006,0.026356
6,10,0.342296,0.343533,0.057581,0.20605,0.206637,0.605518,0.360222,0.098579,0.947149,1.098898,0.005385,0.636026,0.83178,0.983172,0.058918,0.033335,0.999414,0.946593,0.004585,6.600996,0.983545,0.017978,0.531303,3.179158,0.025614
7,10,0.342267,0.34352,0.057592,0.205235,0.205805,0.601409,0.359072,0.098594,0.945389,1.092631,0.005428,0.615709,0.852596,0.985253,0.068996,0.037322,0.999414,0.944835,0.005519,6.836902,0.985627,0.021639,0.550291,3.077602,0.025816
8,10,0.352104,0.353428,0.06001,0.212681,0.213278,0.612019,0.369133,0.102544,0.951253,1.122514,0.005458,0.605431,0.861581,0.986152,0.082397,0.042195,0.999414,0.950696,0.006343,6.998396,0.986526,0.024872,0.563289,3.026231,0.025962


In [23]:
best_hyperparameters = search_metadata["hyperparameters_best"]
best_hyperparameters

{'topK': 1000,
 'alpha': 0.8855381873258531,
 'beta': 0.7129199046253566,
 'normalize_similarity': True}

In [24]:
exception_list = search_metadata["exception_list"]
any(exception_list)

False

In [16]:
output_folder_path = "result_experiments/RP3beta_augmented_subgenre_channel/improvedRange"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

n_cases = 50  # 50 with 30% random is a good number
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [11]:
import HyperparameterTuning

In [12]:
import importlib
importlib.reload(HyperparameterTuning)

<module 'HyperparameterTuning' (namespace)>

In [17]:
from functools import partial
import os, multiprocessing

from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

from HyperparameterTuning.run_hyperparameter_search import runHyperparameterSearch_Collaborative

runHyperparameterSearch_Collaborative(RP3betaRecommender,
                                        URM_train = URM_aug_train,
                                        URM_train_last_test = None,
                                        metric_to_optimize = metric_to_optimize,
                                        cutoff_to_optimize = cutoff_to_optimize,
                                        n_cases = n_cases,
                                        n_random_starts = n_random_starts,
                                        evaluator_validation_earlystopping = evaluator_validation,
                                        evaluator_validation = evaluator_validation,
                                        evaluator_test = None,
                                        output_folder_path = output_folder_path,
                                        resume_from_saved = True,
                                        similarity_type_list = None,
                                        parallelizeKNN = True)

SearchBayesianSkopt: Resuming 'RP3betaRecommender' Failed, no such file exists.

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 1096, 'alpha': 1.1940632722337996, 'beta': 0.9432191301698756, 'normalize_similarity': True}
EvaluatorHoldout: Processed 13646 (100.0%) in 41.21 sec. Users per second: 331
SearchBayesianSkopt: New best config found. Config 0: {'topK': 1096, 'alpha': 1.1940632722337996, 'beta': 0.9432191301698756, 'normalize_similarity': True} - results: PRECISION: 0.0254140, PRECISION_RECALL_MIN_DEN: 0.0254487, RECALL: 0.0050193, MAP: 0.0074372, MAP_MIN_DEN: 0.0074487, MRR: 0.0453666, NDCG: 0.0224283, F1: 0.0083829, HIT_RATE: 0.1735307, ARHR_ALL_HITS: 0.0576026, NOVELTY: 0.0099023, AVERAGE_POPULARITY: 0.0180131, DIVERSITY_MEAN_INTER_LIST: 0.9661772, DIVERSITY_HERFINDAHL: 0.9966106, COVERAGE_ITEM: 0.2587076, COVERAGE_ITEM_CORRECT: 0.0549311, COVERAGE_USER: 0.9997070, COVERAGE_USER_CORRECT: 0.1734799, DIVERSITY_GINI: 0

In [18]:
from Recommenders.DataIO import DataIO

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(RP3betaRecommender.RECOMMENDER_NAME + "_metadata.zip")

search_metadata.keys()

dict_keys(['time_on_validation_total', 'time_on_test_avg', 'result_on_validation_df', 'time_df', 'hyperparameters_best_index', 'time_on_test_total', 'algorithm_name_recommender', 'result_on_last', 'hyperparameters_df', 'metric_to_optimize', 'time_on_validation_avg', 'cutoff_to_optimize', 'result_on_test_df', 'result_on_test_best', 'time_on_train_avg', 'time_on_train_total', 'algorithm_name_search', 'exception_list', 'time_on_last_df', 'result_on_validation_best', 'hyperparameters_best'])

In [19]:
hyperparameters_df = search_metadata["hyperparameters_df"]
hyperparameters_df.head(54)

,topK,alpha,beta,normalize_similarity
0,1096,1.194063,0.943219,True
1,1144,0.421889,0.643489,True
2,1131,1.166359,0.109542,True
3,976,0.724371,0.846637,True
4,833,0.027568,0.860464,True
5,832,0.585423,0.86644,True
6,1142,0.06308,0.72254,True
7,1172,0.179832,0.211472,True
8,880,0.321892,0.579206,True
9,1145,0.389597,0.229685,True


In [20]:
result_on_validation_df = search_metadata["result_on_validation_df"]
result_on_validation_df.head(54)

,,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
,cutoff,,,,,,,,,,,,,,,,,,,,,
0,10,0.025414,0.025449,0.005019,0.007437,0.007449,0.045367,0.022428,0.008383,0.173531,0.057603,...,0.054931,0.999707,0.17348,0.04326,9.665428,0.996989,0.169613,0.777956,0.090331,0.047098
1,10,0.354521,0.355586,0.060394,0.214956,0.215433,0.614711,0.371342,0.103207,0.950828,1.129465,...,0.050889,0.999707,0.950549,0.008382,7.358576,0.989141,0.032863,0.592281,2.880222,0.026259
2,10,0.34169,0.34278,0.057798,0.20362,0.204101,0.601631,0.357898,0.098871,0.946724,1.088271,...,0.027576,0.999707,0.946447,0.00479,6.754545,0.985655,0.01878,0.543664,2.924374,0.025949
3,10,0.32699,0.32782,0.054914,0.190114,0.190397,0.56137,0.337909,0.094036,0.930456,1.016249,...,0.082452,0.999707,0.930183,0.019771,8.288702,0.993578,0.077517,0.667146,2.40528,0.027804
4,10,0.176286,0.176717,0.027966,0.082559,0.082693,0.361995,0.180899,0.048274,0.776491,0.541324,...,0.113904,0.999707,0.776264,0.029888,8.805414,0.994036,0.117185,0.708735,1.154213,0.03074
5,10,0.284552,0.285262,0.046877,0.156768,0.157029,0.507311,0.293,0.080494,0.899384,0.878695,...,0.101224,0.999707,0.899121,0.027068,8.816522,0.99536,0.106127,0.709629,1.922137,0.028943
6,10,0.333739,0.334674,0.055737,0.195855,0.196252,0.579131,0.346592,0.095521,0.936611,1.046801,...,0.07099,0.999707,0.936337,0.014925,8.233525,0.994189,0.058519,0.662705,2.442084,0.027024
7,10,0.334171,0.335409,0.055499,0.200842,0.20131,0.602895,0.353118,0.095189,0.942254,1.081462,...,0.021374,0.999707,0.941978,0.002907,6.051566,0.978799,0.011398,0.487082,3.367305,0.025164
8,10,0.352382,0.353571,0.060029,0.213856,0.214356,0.614518,0.370116,0.102583,0.950682,1.127819,...,0.04441,0.999707,0.950403,0.006888,7.10733,0.987252,0.027007,0.572059,2.981823,0.026031


In [21]:
best_hyperparameters = search_metadata["hyperparameters_best"]
best_hyperparameters

{'topK': 800,
 'alpha': 0.8865772367151288,
 'beta': 0.6814897662772657,
 'normalize_similarity': True}

In [9]:
output_folder_path = "result_experiments/RP3beta_augmented_subgenre_channel/improvedRange2"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

n_cases = 50  # 50 with 30% random is a good number
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [10]:
from functools import partial
import os, multiprocessing

from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

from HyperparameterTuning.run_hyperparameter_search import runHyperparameterSearch_Collaborative

runHyperparameterSearch_Collaborative(RP3betaRecommender,
                                        URM_train = URM_aug_train,
                                        URM_train_last_test = None,
                                        metric_to_optimize = metric_to_optimize,
                                        cutoff_to_optimize = cutoff_to_optimize,
                                        n_cases = n_cases,
                                        n_random_starts = n_random_starts,
                                        evaluator_validation_earlystopping = evaluator_validation,
                                        evaluator_validation = evaluator_validation,
                                        evaluator_test = None,
                                        output_folder_path = output_folder_path,
                                        resume_from_saved = True,
                                        similarity_type_list = None,
                                        parallelizeKNN = True)

SearchBayesianSkopt: Resuming 'RP3betaRecommender' Failed, no such file exists.

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 774, 'alpha': 0.1234875473020624, 'beta': 0.3768027067742467, 'normalize_similarity': True}
EvaluatorHoldout: Processed 13646 (100.0%) in 24.85 sec. Users per second: 549
SearchBayesianSkopt: New best config found. Config 0: {'topK': 774, 'alpha': 0.1234875473020624, 'beta': 0.3768027067742467, 'normalize_similarity': True} - results: PRECISION: 0.3430896, PRECISION_RECALL_MIN_DEN: 0.3443884, RECALL: 0.0579592, MAP: 0.2062395, MAP_MIN_DEN: 0.2067587, MRR: 0.6091248, NDCG: 0.3609987, F1: 0.0991659, HIT_RATE: 0.9473106, ARHR_ALL_HITS: 1.1018065, NOVELTY: 0.0053655, AVERAGE_POPULARITY: 0.6415830, DIVERSITY_MEAN_INTER_LIST: 0.8147008, DIVERSITY_HERFINDAHL: 0.9814641, COVERAGE_ITEM: 0.0541558, COVERAGE_ITEM_CORRECT: 0.0321723, COVERAGE_USER: 0.9997070, COVERAGE_USER_CORRECT: 0.9470330, DIVERSITY_GINI: 0.0

In [11]:
from Recommenders.DataIO import DataIO

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(RP3betaRecommender.RECOMMENDER_NAME + "_metadata.zip")

search_metadata.keys()

dict_keys(['time_on_validation_total', 'time_on_test_avg', 'result_on_validation_df', 'time_df', 'hyperparameters_best_index', 'time_on_test_total', 'algorithm_name_recommender', 'result_on_last', 'hyperparameters_df', 'metric_to_optimize', 'time_on_validation_avg', 'cutoff_to_optimize', 'result_on_test_df', 'result_on_test_best', 'time_on_train_avg', 'time_on_train_total', 'algorithm_name_search', 'exception_list', 'time_on_last_df', 'result_on_validation_best', 'hyperparameters_best'])

In [12]:
hyperparameters_df = search_metadata["hyperparameters_df"]
hyperparameters_df.head(54)

,topK,alpha,beta,normalize_similarity
0,774,0.123488,0.376803,True
1,542,0.675107,0.824512,True
2,148,0.549434,0.648374,True
3,543,0.533464,1.03612,True
4,738,0.65419,0.215551,True
5,781,1.173303,0.888605,True
6,680,0.73961,0.61385,True
7,130,0.593125,1.190332,True
8,157,0.484623,0.420398,True
9,212,0.655714,0.131602,True


In [13]:
result_on_validation_df = search_metadata["result_on_validation_df"]
result_on_validation_df.head(54)

,,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
,cutoff,,,,,,,,,,,,,,,,,,,,,
0,10,0.34309,0.344388,0.057959,0.206239,0.206759,0.609125,0.360999,0.099166,0.947311,1.101806,...,0.032172,0.999707,0.947033,0.004147,6.457703,0.981837,0.016261,0.519771,3.217387,0.02552
1,10,0.340957,0.341884,0.05729,0.20131,0.201676,0.582025,0.353094,0.098097,0.940862,1.063717,...,0.070214,0.999707,0.940586,0.016431,8.28885,0.994363,0.064424,0.667158,2.480615,0.027315
2,10,0.361564,0.36291,0.062214,0.219237,0.219806,0.622395,0.378078,0.106162,0.956984,1.14776,...,0.076638,0.999707,0.956703,0.012227,7.535058,0.987194,0.047941,0.606486,2.791755,0.026668
3,10,0.018408,0.018433,0.002686,0.007011,0.007019,0.052505,0.019527,0.004687,0.143265,0.060234,...,0.075475,0.999707,0.143223,0.043535,9.356887,0.993132,0.170691,0.753122,0.02122,0.04663
4,10,0.341778,0.343148,0.057849,0.204752,0.205261,0.604363,0.35902,0.09895,0.946944,1.093949,...,0.027632,0.999707,0.946667,0.003745,6.354575,0.980648,0.014682,0.511471,3.185553,0.025554
5,10,0.25908,0.259317,0.042755,0.136337,0.136407,0.396672,0.254067,0.073398,0.803312,0.727983,...,0.068332,0.999707,0.803077,0.032454,8.944322,0.994411,0.127244,0.719916,1.930653,0.033242
6,10,0.357936,0.359084,0.061378,0.217871,0.218361,0.622855,0.375832,0.104788,0.955591,1.145253,...,0.047456,0.999707,0.955311,0.007579,7.162135,0.986943,0.029717,0.57647,2.95249,0.026183
7,10,0.007834,0.007856,0.001155,0.002835,0.002843,0.024097,0.008404,0.002013,0.06654,0.02611,...,0.03472,0.999707,0.06652,0.032191,9.1021,0.994891,0.126213,0.732615,0.010261,0.047842
8,10,0.364861,0.366279,0.063236,0.221272,0.221964,0.623932,0.38093,0.10779,0.959842,1.154074,...,0.068664,0.999707,0.95956,0.010848,7.47405,0.986302,0.042531,0.601576,2.711529,0.026853


In [14]:
best_hyperparameters = search_metadata["hyperparameters_best"]
best_hyperparameters

{'topK': 100,
 'alpha': 0.6196111425127981,
 'beta': 0.5311875199059363,
 'normalize_similarity': True}

In [19]:
output_folder_path = "result_experiments/RP3beta_augmented_subgenre_channel/improvedRange3"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

n_cases = 50  # 50 with 30% random is a good number
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [10]:
from functools import partial
import os, multiprocessing

from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

from HyperparameterTuning.run_hyperparameter_search import runHyperparameterSearch_Collaborative

runHyperparameterSearch_Collaborative(RP3betaRecommender,
                                        URM_train = URM_aug_train,
                                        URM_train_last_test = None,
                                        metric_to_optimize = metric_to_optimize,
                                        cutoff_to_optimize = cutoff_to_optimize,
                                        n_cases = n_cases,
                                        n_random_starts = n_random_starts,
                                        evaluator_validation_earlystopping = evaluator_validation,
                                        evaluator_validation = evaluator_validation,
                                        evaluator_test = None,
                                        output_folder_path = output_folder_path,
                                        resume_from_saved = True,
                                        similarity_type_list = None,
                                        parallelizeKNN = True)

SearchBayesianSkopt: Resuming 'RP3betaRecommender' Failed, no such file exists.

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 83, 'alpha': 0.9266255017813776, 'beta': 0.8657487311011287, 'normalize_similarity': True}
EvaluatorHoldout: Processed 13646 (100.0%) in 23.31 sec. Users per second: 585
SearchBayesianSkopt: New best config found. Config 0: {'topK': 83, 'alpha': 0.9266255017813776, 'beta': 0.8657487311011287, 'normalize_similarity': True} - results: PRECISION: 0.2698373, PRECISION_RECALL_MIN_DEN: 0.2707689, RECALL: 0.0465854, MAP: 0.1502377, MAP_MIN_DEN: 0.1505995, MRR: 0.5113170, NDCG: 0.2840135, F1: 0.0794537, HIT_RATE: 0.8901510, ARHR_ALL_HITS: 0.8668038, NOVELTY: 0.0062015, AVERAGE_POPULARITY: 0.4648183, DIVERSITY_MEAN_INTER_LIST: 0.8816537, DIVERSITY_HERFINDAHL: 0.9881589, COVERAGE_ITEM: 0.1790243, COVERAGE_ITEM_CORRECT: 0.0524946, COVERAGE_USER: 0.9997070, COVERAGE_USER_CORRECT: 0.8898901, DIVERSITY_GINI: 0.015

In [20]:
from Recommenders.DataIO import DataIO

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(RP3betaRecommender.RECOMMENDER_NAME + "_metadata.zip")

search_metadata.keys()

dict_keys(['time_on_validation_total', 'time_on_test_avg', 'result_on_validation_df', 'time_df', 'hyperparameters_best_index', 'time_on_test_total', 'algorithm_name_recommender', 'result_on_last', 'hyperparameters_df', 'metric_to_optimize', 'time_on_validation_avg', 'cutoff_to_optimize', 'result_on_test_df', 'result_on_test_best', 'time_on_train_avg', 'time_on_train_total', 'algorithm_name_search', 'exception_list', 'time_on_last_df', 'result_on_validation_best', 'hyperparameters_best'])

In [21]:
hyperparameters_df = search_metadata["hyperparameters_df"]
hyperparameters_df.head(100)

,topK,alpha,beta,normalize_similarity
0,83,0.926626,0.865749,True
1,162,0.890641,0.603776,True
2,124,0.853847,0.790955,True
3,7,0.031475,0.564896,True
4,147,0.518477,0.72502,True
...,...,...,...,...
95,59,0.384235,0.568418,True
96,61,0.664183,0.580731,True
97,59,0.403549,0.580985,True
98,59,0.52439,0.567004,True


In [22]:
result_on_validation_df = search_metadata["result_on_validation_df"]
result_on_validation_df.tail(50)

,,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
,cutoff,,,,,,,,,,,,,,,,,,,,,
50,10,0.365323,0.366511,0.063103,0.221357,0.221901,0.620006,0.380544,0.107617,0.957936,1.151284,...,0.072319,0.999707,0.957656,0.013667,7.759686,0.988057,0.053584,0.624566,2.585211,0.027221
51,10,0.322234,0.323381,0.056574,0.190492,0.190991,0.592882,0.341198,0.09625,0.943573,1.046719,...,0.074866,0.999707,0.943297,0.01993,8.818489,0.996366,0.078143,0.709788,1.580247,0.029149
52,10,0.364026,0.365381,0.062827,0.220841,0.221511,0.621998,0.380077,0.107159,0.958742,1.151608,...,0.06656,0.999707,0.958462,0.010851,7.507038,0.986524,0.042544,0.604231,2.676989,0.026929
53,10,0.366386,0.367664,0.063495,0.22327,0.223859,0.624471,0.382838,0.108233,0.958596,1.161183,...,0.078188,0.999707,0.958315,0.015932,7.891831,0.988555,0.062465,0.635202,2.603545,0.027298
54,10,0.367324,0.368617,0.063407,0.223781,0.224367,0.624161,0.38348,0.108146,0.960135,1.162134,...,0.098234,0.999707,0.959853,0.017487,8.01789,0.989489,0.068562,0.645348,2.574621,0.027203
55,10,0.350542,0.351476,0.058889,0.20946,0.209869,0.599146,0.364359,0.100837,0.946724,1.101182,...,0.071876,0.999707,0.946447,0.012302,7.701684,0.989921,0.048232,0.619897,2.734838,0.026629
56,10,0.320761,0.321844,0.055503,0.189831,0.190289,0.591258,0.339894,0.094631,0.942767,1.043507,...,0.087546,0.999707,0.942491,0.028387,9.22981,0.997104,0.111301,0.742894,1.394526,0.029983
57,10,0.367939,0.369187,0.063781,0.224096,0.224669,0.626779,0.384178,0.108717,0.960135,1.164737,...,0.07852,0.999707,0.959853,0.016,7.939665,0.988839,0.062733,0.639052,2.540255,0.027403
58,10,0.354199,0.355424,0.061382,0.215394,0.215909,0.620863,0.372699,0.104631,0.954639,1.137321,...,0.090592,0.999707,0.954359,0.024994,8.760761,0.993295,0.097995,0.705141,1.951402,0.028922


In [23]:
best_hyperparameters = search_metadata["hyperparameters_best"]
best_hyperparameters

{'topK': 59,
 'alpha': 0.38423458369392577,
 'beta': 0.5684177421916174,
 'normalize_similarity': True}

In [2]:
output_folder_path = "result_experiments/RP3beta_augmented_subgenre_channel/noAugmentation"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

n_cases = 50  # 50 with 30% random is a good number
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [15]:
from functools import partial
import os, multiprocessing

from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

from HyperparameterTuning.run_hyperparameter_search import runHyperparameterSearch_Collaborative

runHyperparameterSearch_Collaborative(RP3betaRecommender,
                                        URM_train = URM_train,
                                        URM_train_last_test = None,
                                        metric_to_optimize = metric_to_optimize,
                                        cutoff_to_optimize = cutoff_to_optimize,
                                        n_cases = n_cases,
                                        n_random_starts = n_random_starts,
                                        evaluator_validation_earlystopping = evaluator_validation,
                                        evaluator_validation = evaluator_validation,
                                        evaluator_test = None,
                                        output_folder_path = output_folder_path,
                                        resume_from_saved = True,
                                        similarity_type_list = None,
                                        parallelizeKNN = True)

SearchBayesianSkopt: Resuming 'RP3betaRecommender' Failed, no such file exists.

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 1183, 'alpha': 0.7954742914220528, 'beta': 0.24389715063759615, 'normalize_similarity': True}
EvaluatorHoldout: Processed 13646 (100.0%) in 28.71 sec. Users per second: 475
SearchBayesianSkopt: New best config found. Config 0: {'topK': 1183, 'alpha': 0.7954742914220528, 'beta': 0.24389715063759615, 'normalize_similarity': True} - results: PRECISION: 0.3389125, PRECISION_RECALL_MIN_DEN: 0.3401562, RECALL: 0.0569556, MAP: 0.2034402, MAP_MIN_DEN: 0.2039131, MRR: 0.6067261, NDCG: 0.3571138, F1: 0.0975222, HIT_RATE: 0.9447457, ARHR_ALL_HITS: 1.0915733, NOVELTY: 0.0053521, AVERAGE_POPULARITY: 0.6451448, DIVERSITY_MEAN_INTER_LIST: 0.8112128, DIVERSITY_HERFINDAHL: 0.9811153, COVERAGE_ITEM: 0.0418628, COVERAGE_ITEM_CORRECT: 0.0248076, COVERAGE_USER: 0.9997070, COVERAGE_USER_CORRECT: 0.9444689, DIVERSITY_GINI:

In [5]:
from Recommenders.DataIO import DataIO

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(RP3betaRecommender.RECOMMENDER_NAME + "_metadata.zip")

search_metadata.keys()

dict_keys(['time_on_validation_total', 'time_on_test_avg', 'result_on_validation_df', 'time_df', 'hyperparameters_best_index', 'time_on_test_total', 'algorithm_name_recommender', 'result_on_last', 'hyperparameters_df', 'metric_to_optimize', 'time_on_validation_avg', 'cutoff_to_optimize', 'result_on_test_df', 'result_on_test_best', 'time_on_train_avg', 'time_on_train_total', 'algorithm_name_search', 'exception_list', 'time_on_last_df', 'result_on_validation_best', 'hyperparameters_best'])

In [6]:
hyperparameters_df = search_metadata["hyperparameters_df"]
hyperparameters_df.head(50)

,topK,alpha,beta,normalize_similarity
0,1183,0.795474,0.243897,True
1,1090,0.974972,0.662569,True
2,866,0.543604,0.797979,True
3,1186,0.489489,0.324152,True
4,899,0.262459,0.904735,True
5,915,0.712212,0.752845,True
6,892,0.644734,0.456868,True
7,967,0.6455,0.286568,True
8,1023,0.053857,0.030422,True
9,1183,0.711915,0.609687,True


In [7]:
result_on_validation_df = search_metadata["result_on_validation_df"]
result_on_validation_df.head(50)

,,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
,cutoff,,,,,,,,,,,,,,,,,,,,,
0,10,0.338913,0.340156,0.056956,0.20344,0.203913,0.606726,0.357114,0.097522,0.944746,1.091573,...,0.024808,0.999707,0.944469,0.00361,6.337354,0.981494,0.014562,0.51149,3.166047,0.025182
1,10,0.35993,0.360844,0.061378,0.219894,0.220261,0.622073,0.377411,0.104873,0.953686,1.149496,...,0.044576,0.999707,0.953407,0.008623,7.172574,0.986663,0.034782,0.578901,2.933979,0.025997
2,10,0.346175,0.347112,0.058582,0.20453,0.20487,0.589044,0.358373,0.100207,0.945991,1.079503,...,0.073869,0.999707,0.945714,0.017331,8.378645,0.994775,0.069905,0.676244,2.396553,0.026929
3,10,0.339352,0.340704,0.057038,0.204877,0.205408,0.609181,0.358669,0.097662,0.946724,1.098379,...,0.024475,0.999707,0.946447,0.003341,6.211235,0.979983,0.013478,0.501311,3.275547,0.025004
4,10,0.139491,0.139728,0.023815,0.059386,0.059463,0.277724,0.138628,0.040683,0.683863,0.402413,...,0.142422,0.999707,0.683663,0.053505,9.666559,0.996502,0.215814,0.780192,0.795132,0.034976
5,10,0.358611,0.359532,0.061152,0.21759,0.217939,0.613679,0.374413,0.104487,0.951561,1.136301,...,0.063348,0.999707,0.951282,0.013196,7.903302,0.992093,0.053228,0.637879,2.627934,0.026503
6,10,0.348622,0.349897,0.059327,0.211456,0.21205,0.617665,0.36765,0.101398,0.950682,1.123643,...,0.032283,0.999707,0.950403,0.004587,6.550131,0.982333,0.018501,0.528664,3.158437,0.025315
7,10,0.341441,0.342784,0.057642,0.205176,0.205706,0.608188,0.359456,0.098632,0.946065,1.097472,...,0.026137,0.999707,0.945788,0.003606,6.308795,0.980677,0.014544,0.509185,3.200142,0.025153
8,10,0.322695,0.323708,0.053868,0.187508,0.187886,0.573787,0.336762,0.092324,0.935659,1.021225,...,0.030123,0.999707,0.935385,0.005432,6.956001,0.987074,0.021909,0.561422,2.682667,0.025977
